<a href="https://colab.research.google.com/github/yuu067/MIA-IABD-2425/blob/main/UD03/notebooks/EX3.-analisi_sentiment_imdb_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios UD03_03

# Análisis de sentimientos con Distilbert

En la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) hemos entrenado a Distilbert para clasificar los sentimientos en los textos, utilizando una base de datos de Tweets.

En este ejercicio, queremos usar Distilbert para clasificar las reseñas de películas y el mismo entrenamiento no nos sirve. Por esta razón, utilizaremos el aprendizaje de transferencia para adaptar el modelo a la nueva tarea, utilizando el `DataSet` de las reseñas de películas IMDB, disponibles en Huggingface como un `imdb`.

## Objetivos de la práctica

* Reproduce el proceso visto en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) para entrenar a Distilbert para clasificar los sentimientos en las reseñas de películas.
* Deberá entrenar el modelo (Distilbert u otros transformadores) con el `dataset` de las reseñas de películas IMDB.
* Deberá analizar las predicciones del modelo y compararlo con el modelo entrenado en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb).
* Utilitza un `pipeline` de transformers para hacer predicciones [`zero-shot`](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.ZeroShotClassificationPipeline) sobre reseñas de películas. Comparar los resultados con los del modelo entrenado con el `dataset` de IMDB.
* Escribe una celda con tus comentarios y/o conclusiones finales comparando las dos alternativas realizadas en el ejercicio.

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
%pip install -U transformers datasets evaluate accelerate scikit-learn accuracy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('imdb')

# Mostramos los datos de ejemplo

dataset['train'][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
# importamos el tokenizador de DistilBERT
from transformers import AutoTokenizer

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

# Mostramos un ejemplo de tokenización
tokenizer.tokenize('FC Barcelona is fucked this year')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['fc', 'barcelona', 'is', 'fucked', 'this', 'year']

In [5]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [6]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
import evaluate

accuracy = evaluate.load('accuracy')

In [8]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id_a_etiqueta = {
    0: "NEG",
    1: "POS",
}

etiqueta_a_id = {
    "NEG": 0,
    "POS": 1,
}

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 16
NUM_EPOCHS = 1

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['test'],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.210800,0.191082,0.930360


TrainOutput(global_step=1563, training_loss=0.2622132103029765, metrics={'train_runtime': 1717.3829, 'train_samples_per_second': 14.557, 'train_steps_per_second': 0.91, 'total_flos': 3311684966400000.0, 'train_loss': 0.2622132103029765, 'epoch': 1.0})

In [14]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [30]:
for i in range(1,10):
    pred = classifier(dataset["unsupervised"][i]["text"], candidate_labels=["NEG", "POS"])
    print(i,pred["labels"],pred["scores"],pred)

1 ['NEG', 'POS'] [0.5017515420913696, 0.49824848771095276] {'sequence': 'When I say this is my favourite film of all time, that comment is not to be taken lightly. I probably watch far too many films than is healthy for me, and have loved quite a few of them. I first saw "La Femme Nikita" nearly ten years ago, and it still manages to be my absolute favourite. Why?<br /><br />This is more than an incredibly stylish and sexy thriller. Luc Besson\'s great flair for impeccable direction, fashion, and appropriate usage of music makes this a very watchable film. But it is Anne Parillaud\'s perfect rendering of a complex character who transforms from a heartless killer into a compassionate, vibrant young woman that makes this film beautiful. I can\'t keep my eyes off of her when she is on screen.<br /><br />I have seen several of Luc Besson\'s films including "Subway", "The Professional", and the irritating "Fifth Element", and "Nikita" is without a doubt, far superior to any of these. Althou

In [15]:
dataset["unsupervised"]["text"][0]

'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)'